In [1]:
import os
from glob import glob
from subprocess import check_call
import yaml

import xarray as xr

import util

In [2]:
data_campaign = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6'

with open('cplhist-cases.yml') as fid:
    cplhist_cases = yaml.safe_load(fid)

In [3]:
cplhist_case_list = [d['case'] for d in cplhist_cases.values()]
cplhist_case_list

['b.e21.BHIST.f09_g17.CMIP6-historical.011',
 'b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.102']

In [4]:
def ncrcat_surface_salinity(file_list, file_out, varname):
    """Concatenate SALT files and subset z_t=0"""
    check_call([
        './ncrcat-surface-salinity.sh', ' '.join(file_list), file_out, varname
    ])

In [5]:
%%time
freq = 'day_1'
output_format = 'nc'
clobber = False

assert freq in ['month_1', 'day_1']
assert output_format in ['ieeer8', 'nc']

varname = 'SALT' if freq == 'month_1' else 'SSS'
isel_z = {'z_t': 0} if freq == 'month_1' else {}

for case in cplhist_case_list:
    
    dirout = f"{util.restoring_data_stage_root}/{case}"
    os.makedirs(dirout, exist_ok=True)    
    
    files = sorted(glob(f"{data_campaign}/{case}/ocn/proc/tseries/{freq}/*.{varname}.*.nc"))

    datestr0 = files[0].split('.')[-2]
    datestr1 = files[-1].split('.')[-2]      
    datestr_out = '-'.join([datestr0.split('-')[0], datestr1.split('-')[1]])    
    
    file_out = f"{dirout}/{case}.SSS.{freq[:3]}.{datestr_out}.{output_format}"       
    
    if not os.path.exists(file_out) or clobber:
        if output_format == 'ieeer8':
            ds = xr.open_mfdataset(
                files, 
                decode_cf=False, 
                decode_times=False, 
                decode_coords=False, 
                compat='override',
                coords='minimal',
                chunks={'time': 1},
            )
            with open(file_out, "wb") as fid: # or choose 'w+' mode - read "open()" documentation
                for i in range(ds.sizes['time']):
                    array = ds[varname].isel(time=i, **isel_z).values.astype('>f8')
                    array.tofile(fid)        
        else:
            ncrcat_surface_salinity(files, file_out, varname)

CPU times: user 267 ms, sys: 379 ms, total: 646 ms
Wall time: 1h 43min 34s


In [6]:
ds = xr.open_dataset('/glade/scratch/mclong/cplhist_data/restoring_data/b.e21.BHIST.f09_g17.CMIP6-historical.011/b.e21.BHIST.f09_g17.CMIP6-historical.011.SSS.day.18500101-20141231.nc')
ds.time

<xarray.DataArray 'time' (time: 60225)>
array([cftime.DatetimeNoLeap(1850, 1, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 1, 2, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 1, 3, 12, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2014, 12, 29, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 30, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 31, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 1850-01-01 12:00:00 ... 2014-12-31 12:00:00
Attributes:
    bounds:        time_bound
    cell_methods:  time: mean
    long_name:     time